In [38]:
import numpy as np
import yfinance as yf
import pandas as pd
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from numpy.typing import NDArray
import random

from utils import *

## Data loading

In [39]:
years = 4

# Reading tickers from csv file and converting to yfinanc format
all_tickers = list(pd.read_csv('tickers.csv')['Symbol'].astype(str) + '.OL')

# Downloading data from yahoo finance
data = yf.download(
    all_tickers, 
    period=f'{years}Y'
)['Adj Close'].dropna(
    axis=1, 
    thresh=252 * years - 10
)

print(f'Number of stocks included: {len(data.columns)}')

log_returns = np.log(data/data.shift()).dropna()


[*********************100%%**********************]  341 of 341 completed
Number of stocks included: 200


c:\Users\joneh\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


## Diversifier

In [40]:
portfolio_size = 5

best_score = portfolio_size

best_diversified = None

for i in trange(10000):
    # Random ticker sample:
    ticker_sample = random.sample(log_returns.columns.tolist(), portfolio_size)

    corr_matrix = log_returns[ticker_sample].corr()

    score = np.tril(np.array(corr_matrix), k=-1).mean()

    if score < best_score:
        best_score = score
        best_diversified = corr_matrix

stock_names = {ticker:yf.Ticker(ticker).info['shortName'].title() for ticker in best_diversified.columns.tolist()}

display(best_diversified.rename(columns=stock_names, index=stock_names))
print(f'Best score: {best_score}')

100%|██████████| 10000/10000 [00:04<00:00, 2207.57it/s]


,Black Sea Property,River Tech,Northern Drilling,Adevinta,Romreal
Black Sea Property,1.000000,-0.005987,-0.079982,-0.038368,0.007416
River Tech,-0.005987,1.000000,-0.070540,-0.000269,-0.041438
Northern Drilling,-0.079982,-0.070540,1.000000,0.055094,0.021800
Adevinta,-0.038368,-0.000269,0.055094,1.000000,0.030055
Romreal,0.007416,-0.041438,0.021800,0.030055,1.000000


Best score: -0.00488877721793608


## Portfolio picker

In [41]:
portfolio_size = 5

r = 0.02

weights = np.ones(portfolio_size)/portfolio_size

highest_sharpe = 0
expected_return = 0
p_std = 0
best_weights = np.array([])
best_stocks = []

for i in trange(1000):
    # Random ticker sample:
    ticker_sample = random.sample(log_returns.columns.tolist(), portfolio_size)

    mean_returns = log_returns[ticker_sample].mean() * 252
    cov_matrix = log_returns[ticker_sample].cov() * 252

    optimal_weights = find_OW(
        portfolio_sharpe, 
        cov_matrix, 
        mean_returns, 
        r, 
        n=portfolio_size
    )

    p_sharpe = portfolio_sharpe(
        optimal_weights, 
        cov_matrix, 
        mean_returns, 
        r
    )

    if p_sharpe > highest_sharpe:
        highest_sharpe = p_sharpe
        best_weights = optimal_weights
        best_stocks = ticker_sample
        expected_return = mean_returns @ optimal_weights
        p_std = np.sqrt(portfolio_var(optimal_weights, cov_matrix))


stock_names = {ticker:yf.Ticker(ticker).info['shortName'].title() for ticker in best_stocks}

results_OW = pd.DataFrame(
    data=best_weights.round(4), 
    index=best_stocks,
    columns=['Optimal weights']
).rename(stock_names)

display(results_OW)

results_stats = pd.DataFrame(
    data=[expected_return, p_std, highest_sharpe], 
    index=['Expected return', 'Standard deviation', 'Sharpe ratio'], 
    columns=['Portfolio stats'])

display(results_stats)


100%|██████████| 1000/1000 [00:10<00:00, 91.72it/s]


,Optimal weights
Vistin Pharma,1.0
Sparebank 1 Nord-Norge,0.0
Odfjell Ser. B,0.0
Jæren Sparebank,0.0
Sparebank 1 Sørøst-Norge,0.0


,Portfolio stats
Expected return,0.261070
Standard deviation,0.489425
Sharpe ratio,0.492557
